In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from sklearn.model_selection import KFold
import lightgbm as lgb

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing

import os
import pandas as pd
import numpy as np
import joblib

In [0]:
base = "drive/My Drive/외부/공모전/[기상] AI프렌즈 시즌1 온도 추정 경진대회/"

data = pd.read_csv(base + "train.csv", index_col='id')
X_cols = [col for col in data.columns if 'X' in col]
Y_cols = ["Y{0:02}".format(i) for i in range(18)]

train = data[:4320]
val = data[4320:]

In [4]:
train['Y18'] = train[Y_cols].mean(axis=1)
total_data = pd.concat([train, val], axis=0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [5]:
display(train)

,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,Y00,Y01,Y02,Y03,Y04,Y05,Y06,Y07,Y08,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17,Y18
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,9.7,988.8,1.2,0.6,0.0,1009.3,989.6,12.2,1009.9,1009.8,0.0,0.00,82.4,71.3,0.0,101.1,0.0,56.1,0.3,0.0,91.0,0.0,1002.5,1009.8,1.1,2.3,0.5,1008.7,12.3,1001.6,69.1,8.2,10.7,1010.1,0.00,256.4,0.0,77.2,62.6,0.0,12.5,11.5,11.0,20.0,20.0,10.5,10.0,10.5,10.0,7.0,7.5,7.0,9.0,10.0,9.5,9.0,8.0,9.0,10.666667
1,9.3,988.9,1.7,1.9,0.0,1009.3,989.6,12.1,1010.0,1009.9,0.0,0.00,81.2,121.8,0.0,165.3,0.0,0.0,0.4,0.0,91.9,0.0,1002.5,1009.9,0.0,0.0,0.0,1008.7,12.1,1001.7,70.3,8.3,10.3,1010.1,0.00,215.4,0.0,77.3,63.5,0.0,12.5,11.0,11.0,20.0,20.0,10.0,9.5,10.0,9.5,6.5,7.5,7.0,8.5,10.0,9.5,9.0,7.5,9.0,10.444444
2,9.4,989.0,1.1,2.3,0.0,1009.2,989.7,12.1,1010.1,1010.1,0.0,0.00,86.1,232.0,0.0,144.0,0.0,0.0,0.6,0.0,93.3,0.0,1002.4,1009.8,0.0,0.0,0.0,1008.6,12.1,1001.6,71.5,8.0,9.7,1010.0,0.00,235.2,0.0,77.3,63.9,0.0,12.0,11.0,11.0,19.5,19.5,10.0,9.0,10.0,9.0,6.5,7.5,6.5,8.0,9.5,9.5,8.5,7.5,8.5,10.166667
3,9.4,988.9,1.5,0.7,0.0,1009.2,989.6,12.0,1010.0,1010.0,0.0,0.00,87.7,0.0,0.0,115.9,0.0,294.1,0.1,0.0,93.4,0.0,1002.4,1009.7,0.0,0.0,0.2,1008.7,12.1,1001.5,73.2,7.7,9.4,1010.1,0.00,214.0,0.0,77.5,64.5,0.0,12.0,11.0,11.0,19.5,19.5,9.5,9.0,9.5,9.0,6.0,7.0,6.0,8.0,9.5,9.0,8.5,7.5,8.5,10.000000
4,9.2,988.9,0.8,1.7,0.0,1009.2,989.7,12.0,1010.1,1010.0,0.0,0.00,88.9,0.0,0.0,124.7,0.0,0.0,0.0,0.0,94.1,0.0,1002.4,1009.7,0.0,0.0,0.0,1008.7,12.0,1001.5,74.3,7.4,9.4,1010.1,0.00,174.9,0.0,78.0,65.0,0.0,12.0,10.5,10.5,19.0,19.5,9.5,8.5,9.5,8.5,6.0,7.0,6.0,7.5,9.5,9.0,8.5,7.5,8.5,9.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,19.5,987.8,1.7,0.4,0.0,1007.8,988.0,21.3,1007.8,1007.5,0.0,13.21,92.0,227.5,0.0,304.0,0.0,10.7,0.4,0.0,84.1,0.0,1001.2,1008.2,0.4,328.4,0.6,1006.6,21.6,1000.1,82.3,18.8,19.4,1008.2,16.68,272.9,0.0,82.3,74.3,0.0,22.0,22.5,22.5,26.0,26.0,21.0,20.0,20.5,20.5,19.5,20.0,19.0,20.5,20.5,20.0,20.0,20.5,20.0,21.166667
4316,19.3,987.7,2.1,0.9,0.0,1007.8,988.1,21.3,1008.0,1007.4,0.0,13.21,92.0,259.4,0.0,309.1,0.0,0.7,0.3,0.0,85.7,0.0,1001.2,1008.2,0.2,346.7,1.1,1006.8,21.6,1000.1,85.8,18.6,18.5,1008.4,16.68,315.9,0.0,82.4,74.8,0.0,22.0,22.5,22.5,26.0,25.5,21.0,20.0,20.5,20.5,19.5,20.0,19.0,20.5,20.5,20.0,19.5,20.5,19.5,21.083333
4317,19.5,987.8,0.9,1.3,0.0,1007.8,988.3,21.2,1008.1,1007.5,0.0,13.21,91.9,256.1,0.0,318.6,0.0,54.7,0.9,0.0,85.9,0.0,1001.2,1008.3,0.9,342.1,0.5,1006.8,21.5,1000.2,84.1,18.4,19.1,1008.4,16.68,343.3,0.0,82.8,75.4,0.0,21.5,22.5,22.5,26.0,25.5,21.0,20.0,20.5,20.5,19.5,20.0,19.0,20.0,20.0,19.5,19.5,20.0,19.0,20.916667


In [0]:
def train_model(clf):
  clf.fit(train[X_cols], train['Y18'])

def val_model(clf):
  clf.fit(val[X_cols], val['Y18'])

def get_test_accuracy(clf):
  pred = clf.predict(val[X_cols])
  mae = mean_absolute_error(val['Y18'], pred)

  return mae

def set_model(Model, *args):
  clf = Model(args)
  train_model(clfclf)
  print(get_test_accuracy(reg))

def set_submit(clf, name):
  val_model(clf)
  get_submission(clf, name)

def get_submission(model, name="None"):
  submit = pd.read_csv(base + 'sample_submission.csv')
  test = pd.read_csv(base + "test.csv")
  submit['Y18'] = model.predict(test[X_cols])

  submit.to_csv(base + name + ".csv", index=None)

def lgb_cv(dataset, param):
  data = lgb.Dataset(dataset[X_cols], label=dataset['Y18'])

  cv_result = lgb.cv(
      param,
      data,
      num_boost_round=99999,
      nfold = 5,
      early_stopping_rounds=10,
      stratified=False,
      verbose_eval=10
  )

def lgb_train(dataset, param, cv):
  data = lgb.Dataset(dataset[X_cols], label=dataset['Y18'])

  lgb_model = lgb.train(
    param,
    data,
    num_boost_round=len(cv['l1-mean'])
  )

  return lgb_model

In [0]:
lgb_train = lgb.Dataset(train[X_cols], label=train['Y18'])

lgb_param = {
    'objective': 'regression',
    'metrics': 'mae',
    'learning_rate' : 0.01
}

cv_result = lgb.cv(
    lgb_param,
    lgb_train,
    num_boost_round=99999,
    nfold = 5,
    early_stopping_rounds=10,
    stratified=False,
    verbose_eval=10
)

[10]	cv_agg's l1: 5.60145 + 0.051936
[20]	cv_agg's l1: 5.0815 + 0.0462537
[30]	cv_agg's l1: 4.61086 + 0.0421005
[40]	cv_agg's l1: 4.18541 + 0.0377196
[50]	cv_agg's l1: 3.8008 + 0.0335378
[60]	cv_agg's l1: 3.45312 + 0.030527
[70]	cv_agg's l1: 3.14036 + 0.0275579
[80]	cv_agg's l1: 2.85807 + 0.0259144
[90]	cv_agg's l1: 2.60388 + 0.0239842
[100]	cv_agg's l1: 2.37434 + 0.0226707
[110]	cv_agg's l1: 2.16805 + 0.0218772
[120]	cv_agg's l1: 1.98321 + 0.0208235
[130]	cv_agg's l1: 1.81763 + 0.0203159
[140]	cv_agg's l1: 1.66855 + 0.0200058
[150]	cv_agg's l1: 1.53474 + 0.0191125
[160]	cv_agg's l1: 1.41492 + 0.0187116
[170]	cv_agg's l1: 1.3074 + 0.0184807
[180]	cv_agg's l1: 1.21137 + 0.0186791
[190]	cv_agg's l1: 1.12544 + 0.0196017
[200]	cv_agg's l1: 1.0491 + 0.0209955
[210]	cv_agg's l1: 0.981278 + 0.0218436
[220]	cv_agg's l1: 0.920877 + 0.0225518
[230]	cv_agg's l1: 0.866943 + 0.0227645
[240]	cv_agg's l1: 0.81911 + 0.0226876
[250]	cv_agg's l1: 0.776881 + 0.0223115
[260]	cv_agg's l1: 0.739249 + 0.0218

In [0]:
lgb_model = lgb.train(
    lgb_param,
    lgb_train,
    num_boost_round=len(cv_result['l1-mean'])
)
print(get_test_accuracy(lgb_model))

1.9149885683678884


In [0]:
lgb_total = lgb.Dataset(total_data[X_cols], label=total_data['Y18'])

lgb_param = {
    'objective': 'regression',
    'metrics': 'mae',
    'learning_rate' : 0.01
}

cv_result = lgb.cv(
    lgb_param,
    lgb_train,
    num_boost_round=99999,
    nfold = 5,
    early_stopping_rounds=10,
    stratified=False,
    verbose_eval=10
)

[10]	cv_agg's l1: 5.60145 + 0.051936
[20]	cv_agg's l1: 5.0815 + 0.0462537
[30]	cv_agg's l1: 4.61086 + 0.0421005
[40]	cv_agg's l1: 4.18541 + 0.0377196
[50]	cv_agg's l1: 3.8008 + 0.0335378
[60]	cv_agg's l1: 3.45312 + 0.030527
[70]	cv_agg's l1: 3.14036 + 0.0275579
[80]	cv_agg's l1: 2.85807 + 0.0259144
[90]	cv_agg's l1: 2.60388 + 0.0239842
[100]	cv_agg's l1: 2.37434 + 0.0226707
[110]	cv_agg's l1: 2.16805 + 0.0218772
[120]	cv_agg's l1: 1.98321 + 0.0208235
[130]	cv_agg's l1: 1.81763 + 0.0203159
[140]	cv_agg's l1: 1.66855 + 0.0200058
[150]	cv_agg's l1: 1.53474 + 0.0191125
[160]	cv_agg's l1: 1.41492 + 0.0187116
[170]	cv_agg's l1: 1.3074 + 0.0184807
[180]	cv_agg's l1: 1.21137 + 0.0186791
[190]	cv_agg's l1: 1.12544 + 0.0196017
[200]	cv_agg's l1: 1.0491 + 0.0209955
[210]	cv_agg's l1: 0.981278 + 0.0218436
[220]	cv_agg's l1: 0.920877 + 0.0225518
[230]	cv_agg's l1: 0.866943 + 0.0227645
[240]	cv_agg's l1: 0.81911 + 0.0226876
[250]	cv_agg's l1: 0.776881 + 0.0223115
[260]	cv_agg's l1: 0.739249 + 0.0218

In [0]:
lgb_model = lgb.train(
    lgb_param,
    lgb_total,
    num_boost_round=len(cv_result['l1-mean'])
)
get_submission(lgb_model, "lightbgm_Y")